<a href="https://colab.research.google.com/github/compartia/nlp_tools/blob/case-normal/Build_word_cases_statistics_ipnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
GLOBALS__={}

# Build word cases statistics (for case normalisation)

## MAIN, init, load code

In [0]:
#@title  { form-width: "300px", display-mode: "form" }
import os

!pip install sentencepiece
import sentencepiece as spm




# ''' AZ:-IMPORT CODE GITHUB----------------------------------------------😺------ '''
import sys
# ====================================
# ====================================
_git_branch = "tensorflow-model"  # @param {type:"string"}
# ====================================
# ====================================


def _init_import_code_from_gh():
  if 'GLOBALS__' not in globals():
    print('adding global GLOBALS__')
    global GLOBALS__
    GLOBALS__ = {}

  if '_init_import_code_from_gh' in GLOBALS__:
    print('👌 code already imported from GitHub!')
    return

  import subprocess
  def exec(x):
    r = subprocess.check_output(x, shell=True)
    r = r.decode('unicode-escape').encode('latin1').decode('utf8')
    print(r)

  print(f"fetching code from GitHub.....{_git_branch}")
  try:
    exec('rm -r nlp_tools')
  except:
    pass
  exec(f'git clone --single-branch --branch {_git_branch} https://github.com/compartia/nlp_tools.git nlp_tools')

  print('🦊 GIT revision:')
  exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

  sys.path.insert(0, 'nlp_tools')

  # self-test

  print('installing antiword...')
  exec('sudo apt-get install antiword')

  print('installing docx2txt...')
  exec("pip install docx2txt")

  GLOBALS__['_init_import_code_from_gh'] = True

  ''' AZ:-------------------------------------------------IMPORT CODE GITHUB-😺---'''
  print('❤️ DONE importing Code fro GitHub')


# AZ:-INIT ELMO-----------------------------------------------------------------------------------

import tensorflow_hub as hub

#
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML


 

# AZ:- THE CODE----------------------------------------------------------------------------------

def _init_the_code(reset=False):
  if '_init_the_code' in GLOBALS__ and not reset:
    print('👌 Code is alredy imported!')
    return

  from renderer import HtmlRenderer
  from renderer import to_multicolor_text
  from renderer import known_subjects_dict

  from structures import ContractSubject
  from contract_parser import ContractDocument3

  from ml_tools import ProbableValue

  from legal_docs import LegalDocument
  from renderer import as_warning, as_headline_3, as_offset, as_smaller

  class DemoRenderer(HtmlRenderer):

    def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
      display(HTML(html))

    def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
      return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

    def render_multicolor_text(self, tokens, vectors, colormap, min_color=None, _slice=None):
      display(HTML(to_multicolor_text(tokens, vectors, colormap, min_color=min_color, _slice=_slice)))

     
     

     
  GLOBALS__['renderer'] = DemoRenderer()

  GLOBALS__['_init_the_code'] = True
  print("❤️ DONE initializing the code")

  # AZ:-------------------------------------------------Init Protocols context===


def read_doc(fn):
  import docx2txt, sys, os

  text = ''
  try:
    text = docx2txt.process(fn)

  except:
    print("Unexpected error:", sys.exc_info())
    os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
    with open(fn + '.txt') as f:
      text = f.read()

  return text

def interactive_upload(filetype):
  from google.colab import files
  import docx2txt

  print(f'Please select "{filetype}" .docx file:')
  uploaded = files.upload()
  docs = []
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

    with open(fn, "wb") as df:
      df.write(uploaded[fn])
      df.close()

    # extract text

    text = ''
    try:
      text = docx2txt.process(fn)
    except:
      print("Unexpected error:", sys.exc_info())
      os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
      with open(fn + '.txt') as f:
        text = f.read()
    print("Символов в оригинальном документе:", len(text))
    docs.append(text)
    return docs

  
# AZ:- ENDO OF THE THE CODE------------------------------------------------XXXX


# 1.
_init_import_code_from_gh()
 
# 3.
_init_the_code(True)



## 📂👤Load files from GDrive

In [0]:

from google.colab import auth
from google.colab import drive
 

drive.mount('/content/gdrive', force_remount=True)


import glob
def read_documents(filename_prefix):
  texts = {}
  for file in glob.glob(filename_prefix+"*.doc"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.doc FILE!!', file)

  for file in glob.glob(filename_prefix+"*.docx"):
    try:
      text = read_doc(file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.docx FILE!!', file)
      
  return texts

contracts_filename_prefix='/content/gdrive/My Drive/GazpromOil/Contracts/'
contracts = read_documents(contracts_filename_prefix)

charters_filename_prefix='/content/gdrive/My Drive/GazpromOil/Charters/'
charters = read_documents(charters_filename_prefix)

protocols_filename_prefix='/content/gdrive/My Drive/GazpromOil/Protocols/'
protocols = read_documents(protocols_filename_prefix)


assert len(contracts) > 0
assert len(charters) > 0
assert len(protocols) > 0


In [0]:
TEXT = '\n\n'.join(charters.values()) + '\n\n'.join(contracts.values()) + '\n\n'.join(protocols.values())
print(TEXT[1000:2000].strip())



In [0]:
from contract_parser import ContractDocument3


doc = ContractDocument3(TEXT)
doc.parse()
TOKENS = doc.tokens_cc



In [0]:
import operator
  
counter = {}
for t in TOKENS:
  if t.lower() not in counter:
    counter[t.lower()] = {}
  record = counter[t.lower()]
  if t not in record:
    record[t]=1
  else:
    record[t]+=1
    
    
# print(counter)


best_cases={}
for key in counter:
  stats_record = counter[key]
  most_popular =  max(stats_record.items(), key=operator.itemgetter(1))[0] 

  if stats_record[most_popular]>10: #sort of statistically meaninful number of occurenccies
    print( f'{stats_record[most_popular]} \t {most_popular}'  )
    best_cases[key] = most_popular

## Save pickle

In [0]:
import pickle

_path = '/content/gdrive/My Drive/GazpromOil/'

with open(_path+'word_cases_stats.pickle', 'wb') as handle:
  pickle.dump(best_cases, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [0]:
with open(_path+'word_cases_stats.pickle', 'rb') as handle:
  best_cases_read = pickle.load(handle)
  
  
  
best_cases_read ['преимущественного']
 

## Test it

In [0]:
from documents import Tokens
def normalize_text_case(tokens:Tokens, replacements_map):
  result=[]
  for t in tokens:
    key=t.lower()
    if key in replacements_map:
      result.append(replacements_map[key])
    else:
      result.append(key)
  return result


doc = ContractDocument3( list(contracts.values())[0] )
doc.parse()

from text_tools import untokenize
a = untokenize ( normalize_text_case(doc.tokens, best_cases_read)      )
print(a)
  